In [1]:
import pandas as pd
import urllib.request
import xml.etree.ElementTree as ET

# berlin
# boundingbox=[12.8389,52.3030,13.9348,52.7246]
# charlottenburg
# boundingbox = [13.2457, 52.5045, 13.3470, 52.5288]
# westend
boundingbox = [13.2521, 52.5123, 13.2820, 52.5237]
# r= requests.get("http://overpass-api.de/api/map?bbox=13.2457,52.5045,13.3470,52.5288")
#TODO URL an boundingbox anpassen
url="https://overpass-api.de/api/interpreter?data=node%5B%22surface%22%5D%2852%2E494853137273%2C13%2E242473602295%2C52%2E538935532664%2C13%2E321695327759%29%3Bway%5B%22surface%22%5D%5B%22highway%22%5D%2852%2E494853137273%2C13%2E242473602295%2C52%2E538935532664%2C13%2E321695327759%29%3B%28%2E%5F%3B%3E%3B%29%3Bout%3B%0A"

get Data

In [2]:

response = urllib.request.urlopen(url)
xml = response.read()
response.close()

parse Data

In [3]:
root = ET.fromstring(xml)
#root = tree.getroot()


create surface Dataframe

In [5]:

nodes_surface = list()
for way in root.findall('way'):
    for child in list(way):
        if(child.attrib.get('k')=='surface'):
            #print(child.attrib.get('v'))
            for node in way.findall('nd'):
                nodes_surface.append([child.attrib.get('v'), way.attrib.get('id'), node.attrib.get('ref')])
                
df_surface_node=pd.DataFrame(nodes_surface,columns=['surface','wayId','nodeId'])
df_surface_node

,surface,wayId,nodeId
0,asphalt,4067882,27560124
1,asphalt,4067882,282898608
2,asphalt,4067882,282898609
3,asphalt,4067882,16541461
4,asphalt,4067883,29030447
5,asphalt,4067883,29030449
6,asphalt,4067883,29030445
7,asphalt,4067883,1221501673
8,asphalt,4067883,29030450
9,asphalt,4067883,16541480


create list of node

In [6]:
node_list = list()
for way in root.findall('node'):
    node_list.append([way.attrib.get('id'), way.attrib.get('lat'), way.attrib.get('lon')])
df_nodes = pd.DataFrame(node_list,columns=['nodeId','lat','lon'])
df_nodes

,nodeId,lat,lon
0,458322,52.5037711,13.2824173
1,458323,52.5042930,13.2827047
2,458324,52.5051202,13.2828139
3,458325,52.5054972,13.2828394
4,458326,52.5062206,13.2830150
5,458327,52.5064340,13.2831319
6,458328,52.5067215,13.2833130
7,458329,52.5086169,13.2846003
8,458330,52.5115569,13.2862166
9,458331,52.5121595,13.2863692


In [7]:
df_joined_list = df_surface_node.merge(df_nodes,on='nodeId',how='left')

In [9]:
df_joined_list

,surface,wayId,nodeId,lat,lon
0,asphalt,4067882,27560124,52.5153490,13.2493683
1,asphalt,4067882,282898608,52.5154934,13.2496738
2,asphalt,4067882,282898609,52.5155538,13.2498976
3,asphalt,4067882,16541461,52.5156244,13.2502680
4,asphalt,4067883,29030447,52.5077092,13.2615167
5,asphalt,4067883,29030449,52.5075733,13.2617896
6,asphalt,4067883,29030445,52.5073287,13.2623108
7,asphalt,4067883,1221501673,52.5071811,13.2626230
8,asphalt,4067883,29030450,52.5070144,13.2629412
9,asphalt,4067883,16541480,52.5067636,13.2633591


In [228]:
#count nodes per way
df_surface_node.groupby('wayId').count()

,surface,nodeId
wayId,,
102698235,4,4
102943813,2,2
102943816,2,2
102945288,9,9
103258836,11,11
103453538,9,9
106652853,5,5
106652861,2,2
106652862,2,2
